In [1]:
import torch
from torch import nn 
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [2]:
at = torch.tensor([1,2,3])
torch.less(at,3)*at

tensor([1, 2, 0])

In [3]:
latent_size = 10
class upper_softmax(nn.Module):
    def __init__(self, slope = 1):
       super().__init__()
    def forward(self,x):
        x = torch.softmax(x)
        x = torch.less(x,1/x.shape[0])*x + torch.greater_equal(x,1/x.shape[0])
    
class Generator:
    def __init__(self, latent_size):
        self.model = nn.Sequential(
            nn.Linear(latent_size, latent_size),
            nn.Linear(latent_size, latent_size),
            nn.Linear(latent_size, latent_size),
            nn.Linear(latent_size, latent_size),
            upper_softmax()
        )
        if torch.cuda.is_available():
            device = torch.device('cuda')
            self.model.to(device)
        elif torch.backends.mps.is_available(): #Mac silicon support 
            device = torch.device('mps')
            self.model.to(device)



In [11]:
generator = Generator(latent_size).model
next(generator.parameters()).is_mps

True

In [4]:
import pandas as pd
import numpy as np 
mean = [1,1,0,0,0,0,0,0,2,1]
cov = [[1,1,0,0,0,0,0,0,0,0],[1,1,0,0,0,0,0,0,0,0],[0,0,1,1,1,0,0,0,0,0],[0,0,1,1,1,0,0,0,0,0],[0,0,1,1,1,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,0],[0,0,0,0,0,0,1,0,0,0],
               [0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,0,0,1]]
X_data = pd.DataFrame(np.random.multivariate_normal(mean,cov,200))

: 

In [31]:
from torch.utils.data import DataLoader
data_loader = DataLoader(X_data.to_numpy(), batch_size=9, drop_last=True,pin_memory=True,shuffle=True)

In [5]:
noise_tensor = torch.Tensor(9, latent_size)
noise_tensor.normal_()

tensor([[-0.5559, -1.3276, -1.3735,  0.2391,  0.7134,  0.5364,  0.8910,  0.5169,
         -1.0035, -0.1991],
        [-1.2483, -0.9078, -0.4448, -0.7850, -0.3710,  2.8379, -1.6062, -0.0926,
         -0.8010, -1.6289],
        [-1.1286,  0.1516, -0.3600,  2.0430,  0.5107,  0.3054, -0.8726,  0.0783,
         -1.5533, -0.1265],
        [ 0.9682, -0.8882,  2.1182,  0.5816, -0.5112,  0.0103,  1.2024,  0.9760,
          0.3082, -1.4826],
        [ 0.8171, -1.9253,  0.4690,  0.0894, -1.4751,  1.7497, -1.4958, -0.6230,
         -1.8500, -1.5287],
        [-2.3415, -0.7552,  0.9324,  0.6024,  0.9653, -1.8736,  1.5429, -0.9879,
         -2.0882, -0.4242],
        [-1.9361,  0.4231,  1.4859, -1.7767,  0.2472,  0.6116,  1.0092, -1.2521,
          1.6974,  0.1818],
        [-0.0115, -0.4520,  1.7451,  0.1736, -0.8215,  0.1052, -0.5591,  0.1535,
         -0.2808, -2.3001],
        [ 0.4738,  0.3066, -0.0194,  0.0728,  2.4354,  0.2141,  0.0477,  1.3887,
          1.0828, -1.4216]])

In [59]:
for batch in data_loader:
    #print(batch.__class__); print(batch.dtype)
    print(batch.dtype)
    
    print(batch.to(torch.float32).dtype)
    print(batch.dtype)
    batch = batch.to(torch.float32)
    batch = batch.to('mps')
    print(batch)

torch.float64
torch.float32
torch.float64
tensor([[ 0.7713,  0.7713,  1.2690,  1.2690,  1.2690, -0.3654,  0.4260, -0.8688,
          2.5935,  1.6772],
        [-0.2923, -0.2923,  0.0543,  0.0543,  0.0543,  0.3345,  0.1348,  0.5190,
          1.8297,  1.5603],
        [ 1.9971,  1.9971,  0.6497,  0.6497,  0.6497,  1.3455, -0.3340, -0.2102,
          1.3130, -0.4293],
        [ 0.5629,  0.5629, -0.7599, -0.7599, -0.7599, -0.2512, -0.2023,  0.7319,
          3.1342,  0.3600],
        [-0.0511, -0.0511, -0.0034, -0.0034, -0.0034, -0.2722,  0.2446,  0.5867,
          1.7243,  1.6516],
        [ 0.2987,  0.2987, -0.4994, -0.4994, -0.4994,  1.3535,  0.9083, -1.9821,
          0.4681, -0.2908],
        [ 0.6918,  0.6918,  0.6176,  0.6176,  0.6176, -1.6453, -0.4352, -0.3750,
          1.9053, -0.1475],
        [ 2.7827,  2.7827, -0.1727, -0.1727, -0.1727, -0.5017,  1.0464,  0.6341,
          1.5293, -1.0974],
        [ 1.0483,  1.0483, -0.6048, -0.6048, -0.6048,  0.1219,  0.3811,  0.2824,
     

In [1705]:
def distribution():
    random_number = np.random.uniform(1)
    if random_number > 0.02440 + 0.00238:
        return np.array([0,1,0,1,1,0,1,1,1,0])
    elif random_number <= 0.02440 + 0.00238 and random_number > 0.02440:
        return np.array([0,1,0,1,1,0,0,1,1,0])
    else:
        return np.array([0,1,0,1,1,1,1,1,1,0])
def generator(num):
    return np.array([distribution() for i in range(num)])    

In [1815]:
torch.ones([2,2])

tensor([[1., 1.],
        [1., 1.]])

In [1806]:
import torch_two_sample as tts

n_sample = 100
mean = [1,1,0,0,0,0,0,0,2,1]
cov = [[1,1,0,0,0,0,0,0,0,0],[1,1,0,0,0,0,0,0,0,0],[0,0,1,1,1,0,0,0,0,0],[0,0,1,1,1,0,0,0,0,0],[0,0,1,1,1,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,0],[0,0,0,0,0,0,1,0,0,0],
               [0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,0,0,1]]
X_sample = torch.mps.Tensor(pd.DataFrame(np.random.multivariate_normal(mean,cov,1000)).sample(n_sample).to_numpy()).to('mps:0')
selected = torch.mps.Tensor(generator(n_sample)).to('mps:0')
uX_data =  selected*X_sample + torch.mean(X_sample,dim=0)*(torch.less(selected,1))
mmd = tts.MMDStatistic(n_sample, n_sample) 
mmd_val, distances = mmd(torch.mps.Tensor(X_sample).to('mps:0'),uX_data,alphas=[0.01], ret_matrix=True)

print(f'pval:{mmd.pval(distances)}; distance:{mmd_val}')


pval:0.4869999885559082; distance:-0.0002789497375488281


In [2]:
import numpy as np 

In [14]:
Xx = np.triu([
        [1,0,10,10,0,0,0,0,0,0], 
        [0,1,0,0,0,0,0,0,0,0],
        [0,0,1,10,0,0,0,0,0,0],
        [0,0,0,1,0,0,0,0,0,0],
        [0,0,0,0,1,0,0,0,0,0],
        [0,0,0,0,0,1,0,0,0,0],
        [0,0,0,0,0,0,1,0,0,0],
        [0,0,0,0,0,0,0,1,0,0],
        [0,0,0,0,0,0,0,0,1,0],
        [0,0,0,0,0,0,0,0,0,1]
        ])
Xx = Xx + np.transpose(Xx) - np.diag(np.diag(Xx))
Xx

array([[ 1,  0, 10, 10,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [10,  0,  1, 10,  0,  0,  0,  0,  0,  0],
       [10,  0, 10,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1]])

In [165]:
dim_subspace = 3
dim_space = 10
sigma = 10

def correlation_matrix_generator( dim_subspace, sigma, dim_space = 10, return_subspace = False):
    '''Create a correlation matrix
    
    This functions creates, give a dimensionality of a whole space, the desired subspace, and the covariance (sigma), a simetric covariance distance with a random subspace embedded
    in the correlations.
    '''
    corr_vector = np.sort(np.random.choice(range(dim_space - 1),dim_subspace,replace=False))
    x_matrix = np.diag(np.ones(dim_space))

    for i, feature in enumerate(corr_vector):  
        if i < corr_vector.size - 1:  
            for j in range(corr_vector.size - 1 - i):    
                x_matrix[feature, corr_vector[i+j+1]] = sigma

    if return_subspace == True: return x_matrix + np.transpose(x_matrix) - np.diag((np.ones(dim_space))), corr_vector
    else: return x_matrix + np.transpose(x_matrix) - np.diag((np.ones(dim_space)))
correlation_matrix_generator(3,10,10,return_subspace=False)

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1., 10.,  0.,  0.,  0.,  0., 10.,  0.],
       [ 0.,  0., 10.,  1.,  0.,  0.,  0.,  0., 10.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0., 10., 10.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [170]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

is_pos_def(correlation_matrix_generator(3,10,10,return_subspace=False))

False

In [178]:
x=correlation_matrix_generator(3,10,10,return_subspace=False)
print(x)
print(f"\n")
print(np.transpose(x))

np.matmul(x,np.transpose(x))

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0. 10. 10.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0. 10.  0.  0.  1. 10.  0.  0.  0.  0.]
 [ 0. 10.  0.  0. 10.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0. 10. 10.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0. 10.  0.  0.  1. 10.  0.  0.  0.  0.]
 [ 0. 10.  0.  0. 10.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


array([[  1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0., 201.,   0.,   0., 120., 120.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0., 120.,   0.,   0., 201., 120.,   0.,   0.,   0.,   0.],
       [  0., 120.,   0.,   0., 120., 201.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.]])